In [1]:
#Unit test for BirthProcess.py
import os
import sys

sys.path.append('../') #When modules are in parallel locations, going one path level above makes one module visible to the other
from tabularepimdl.BirthProcess import BirthProcess

import unittest
from unittest.mock import patch
import numpy as np
import pandas as pd

class TestBirthProcess(unittest.TestCase):

    def setUp (self): #prepare data values for below tests
        self.rate = 0.00173 #The global birth rate in 2024 is 17.299 births per 1,000 people
        self.start_state_sig_dict = {'state1':1, 'state2':2}
        self.start_state_sig_df =  pd.DataFrame([self.start_state_sig_dict])
        self.N_value = 1000
        self.current_state = pd.DataFrame({'N': [self.N_value]})
        self.dt = 1.0

    def test_initialization_with_dict(self): #start_date_sig is dictionary type
        bp_dict = BirthProcess(rate=self.rate, start_state_sig=self.start_state_sig_dict, stochastic=False)
        self.assertEqual(bp_dict.rate, self.rate, "birthprocess rate is not euqal to {}".format(self.rate))
        pd.testing.assert_frame_equal(bp_dict.start_state_sig, self.start_state_sig_df, check_dtype=True) #data type should be the same unless it is not required
        self.assertFalse(bp_dict.stochastic, "birthprocess stochastic boolean is not {}".format(bp_dict.stochastic))

    def test_initialization_with_dataframe(self): #start_date_sig is dataframe type
        bp_df = BirthProcess(rate=self.rate, start_state_sig=self.start_state_sig_df, stochastic=True)
        self.assertEqual(bp_df.rate, self.rate, 'birthprocess rate is not equal to {}'.format(self.rate))
        pd.testing.assert_frame_equal(bp_df.start_state_sig, self.start_state_sig_df, check_dtype=True) #data type should be the same unless it is not required
        self.assertTrue(bp_df.stochastic, "birthprocess stochastic boolean is not {}".format(bp_df.stochastic))

    def test_get_deltas_deterministic(self): #the process is deterministic
        bp_deterministic = BirthProcess(rate=self.rate, start_state_sig=self.start_state_sig_dict, stochastic=False)
        expected_births = self.start_state_sig_df.assign(N = self.N_value*(1-np.exp(-self.dt*self.rate)))
        #print('expected_brith in deterministic is: ', expected_births) #print check
        actual_births = bp_deterministic.get_deltas(self.current_state)
        pd.testing.assert_frame_equal(actual_births, expected_births)

    @patch('numpy.random.poisson') #mock object numpy.random.poisson
    def test_get_deltas_stochastic(self, mock_poisson): #the process is stochastic, mock_poisson mocks np.random.poisson
        mock_poisson.return_value = 10 #mocked drawn sample from poisson distribution returns 10
        bp_stochatic = BirthProcess(rate=self.rate, start_state_sig=self.start_state_sig_dict, stochastic=True)
        expected_births = self.start_state_sig_df.assign(N=10)
        #print('expected_brith in stochastic is: ', expected_births) #print check
        actual_births = bp_stochatic.get_deltas(self.current_state) #get_detlas will invoke mocked poisson function and return 10
        pd.testing.assert_frame_equal(actual_births, expected_births)
        mock_poisson.assert_called_once_with(self.N_value * (1 - np.exp(-self.dt * self.rate))) #Assert that the mock was called exactly once and that call was with the specified arguments.
    
    def test_to_yaml(self):
        bp = BirthProcess(rate=self.rate, start_state_sig=self.start_state_sig_dict, stochastic=True)
        result_yaml = bp.to_yaml()
    
        # Extract the DataFrame from the result yaml and the remaining dictionaries
        result_start_state_sig = result_yaml['tabularepimdl.BirthProcess'].pop('start_state_sig') #remove dataframe from result_yaml and put the df into a new object
        expected_start_state_sig = self.start_state_sig_df
    
        # Compare the remaining parts of the dictionary
        expected_yaml = {
            'tabularepimdl.BirthProcess': {
            'rate': self.rate,
            'stochastic': True
            }
        }

        #print(result_yaml)
        self.assertEqual(result_yaml, expected_yaml)
    
        # Compare the DataFrames separately
        pd.testing.assert_frame_equal(result_start_state_sig, expected_start_state_sig)


if __name__ == "__main__":
    unittest.main(argv=[''], verbosity=2, exit=False)
        

test_get_deltas_deterministic (__main__.TestBirthProcess.test_get_deltas_deterministic) ... ok
test_get_deltas_stochastic (__main__.TestBirthProcess.test_get_deltas_stochastic) ... ok
test_initialization_with_dataframe (__main__.TestBirthProcess.test_initialization_with_dataframe) ... ok
test_initialization_with_dict (__main__.TestBirthProcess.test_initialization_with_dict) ... ok
test_to_yaml (__main__.TestBirthProcess.test_to_yaml) ... ok

----------------------------------------------------------------------
Ran 5 tests in 0.012s

OK
